# RAG using langchain and Azure AI

In this example we will use langchain and Azure AI to implement a retriver (as we created in the previous task).

LangChain is an open source framework for building applications based on large language models (LLMs). LangChain provides tools and abstractions to improve the customization, accuracy, and relevancy of the information the models generate. For example, developers can use LangChain components to build new prompt chains or customize existing templates. LangChain also includes components that allow LLMs to access new data sets without retraining.

Azure AI is basically OpenAI as a Service

In [ ]:
%%html

<iframe width="560" height="315" src="https://www.youtube.com/embed/P3MAbZ2eMUI?si=UMXfTuXOmll-oajq" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

## First we setup and check for needed environment variables and config

In [ ]:
import os
import dotenv

dotenv.load_dotenv()

os.environ["AZURESEARCH_FIELDS_ID"] = "chunk_id"
os.environ["AZURESEARCH_FIELDS_CONTENT"] = "chunk"
os.environ["AZURESEARCH_FIELDS_CONTENT_VECTOR"] = "vector"
os.environ["AZURESEARCH_FIELDS_TAG"] = "metadata"

from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

if not os.getenv("AZURE_OPENAI_ENDPOINT"):
    raise ValueError("Please set the environment variable AZURE_OPENAI_ENDPOINT")

if not os.getenv("AZURE_OPENAI_API_KEY"):
    raise ValueError("Please set the environment variable AZURE_OPENAI_API_KEY")

if not os.getenv("AZURE_EMBEDDINGS_DEPLOYMENT"):
    raise ValueError("Please set the environment variable AZURE_EMBEDDINGS_DEPLOYMENT")

if not os.getenv("AZURE_CHAT_DEPLOYMENT"):
    raise ValueError("Please set the environment variable AZURE_CHAT_DEPLOYMENT")

if not os.getenv("AZURE_SEARCH_ENDPOINT"):
    raise ValueError("Please set the environment variable AZURE_SEARCH_ENDPOINT")

api_version = os.getenv("OPENAI_API_VERSION", "2023-03-15-preview")


# Data
Trenger offentlig data som ikke er scrapet av microsoft!

10 tilfeldigvalgte rovdyr tiltak fra https://www.miljovedtak.no/

In [ ]:
query = "Hva kan du om jerv i snåsa?"
#query = "Fra hvilken dato er den nyeste informasjone du har tilgjengelig?"
#query = "Kan du fortelle meg hva miljø direktoratet syns om svensk jerv?"
#query = "Hvor mange tilak har miljødirektoratet gitt i Lierne de siste månedene og for hvilke arter?"
#query = "What is the worlds smallest animal?"

In [ ]:
_model = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_CHAT_DEPLOYMENT"],
    api_version=api_version
)

In [ ]:
# RAG prompt
rag_template = """Answer the question based only on the following context:
{context}
Question: {question}

answer all questions in Norwegian (nynorsk) and specify where you found your information if it is relevant.

"""

# Pure LLM prompt
llm_template = """
Question: {question}

answer all questions as precisely as possible in Norwegian (nynorsk) and specify where you found your information if it is relevant.
"""

In [ ]:
llm_chain = ChatPromptTemplate.from_template(llm_template) | _model | StrOutputParser()


In [ ]:
llm_chain.invoke(query)

In [ ]:
embeddings = AzureOpenAIEmbeddings(
    deployment=os.environ["AZURE_EMBEDDINGS_DEPLOYMENT"],
    chunk_size=1,
)

In [ ]:
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    azure_search_key=os.environ["AZURE_SEARCH_KEY"],
    index_name="kodekveld-mdir-test", #"alternative_facts"
    embedding_function=embeddings.embed_query,
)

retriever = vector_store.as_retriever()

In [ ]:
vector_store.as_retriever(k=4).invoke("Jerv i snåsa")

In [ ]:
rag_chain = (
        RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
        | ChatPromptTemplate.from_template(rag_template)
        | _model
        | StrOutputParser()
)

In [ ]:
rag_chain.invoke(query)

## Hvor bra er RAG?

Forskningen jeg har sett peker på at det øker nøyaktigheten på modellene betydelig, 80-90% avhengig av hvilken modell som er brukt.

Men modellen vil fortsatt ta feil i blant, og kan fortsatt halusinere (kilder til svarene er viktig!). Modellene har en sterk bias mot "intern" data, så hvis du gir kontekst som motstrider dette vil den som regel velge intern data

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("text-input/true_facts.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

#vector_store.add_documents(documents=docs)